In [1]:
import tensorflow as tf
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.utils import to_categorical

# Ensure TensorFlow is set up correctly
assert tf.__version__ >= "2.0", "This code requires TensorFlow 2.0 or higher."

# Constants
INPUT_SHAPE = (32, 32, 3)
NUM_CLASSES = 10
BATCH_SIZE = 64
EPOCHS = 50
LEARNING_RATE = 0.001

def load_and_preprocess_data():
    """
    Loads the CIFAR-10 dataset and preprocesses it for training.
    """
    (x_train, y_train), (x_test, y_test) = cifar10.load_data()

    # Normalize the images to [0, 1] range
    x_train = x_train.astype('float32') / 255.0
    x_test = x_test.astype('float32') / 255.0

    # One-hot encode the labels
    y_train = to_categorical(y_train, NUM_CLASSES)
    y_test = to_categorical(y_test, NUM_CLASSES)

    return x_train, y_train, x_test, y_test


def build_model(input_shape, num_classes):
    """
    Builds a CNN model for CIFAR-10 classification.

    Args:
        input_shape (tuple): Shape of the input images.
        num_classes (int): Number of classes in the dataset.

    Returns:
        model (Sequential): Compiled Keras model.
    """
    model = Sequential([
        Conv2D(32, (3, 3), activation='relu', padding='same', input_shape=input_shape),
        MaxPooling2D((2, 2)),
        Dropout(0.25),

        Conv2D(64, (3, 3), activation='relu', padding='same'),
        MaxPooling2D((2, 2)),
        Dropout(0.25),

        Conv2D(128, (3, 3), activation='relu', padding='same'),
        MaxPooling2D((2, 2)),
        Dropout(0.25),

        Flatten(),

        Dense(512, activation='relu'),
        Dropout(0.5),
        
        Dense(num_classes, activation='softmax')
    ])

    model.compile(optimizer=Adam(learning_rate=LEARNING_RATE),
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])

    return model


def get_callbacks():
    """
    Returns a list of callbacks for training.

    Returns:
        list: List of Keras callbacks.
    """
    early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
    reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=0.00001)
    
    return [early_stopping, reduce_lr]


def train_model(model, x_train, y_train, x_val, y_val, batch_size, epochs):
    """
    Trains the CNN model on the training data.

    Args:
        model (Sequential): Keras model to be trained.
        x_train (numpy.array): Training images.
        y_train (numpy.array): Training labels.
        x_val (numpy.array): Validation images.
        y_val (numpy.array): Validation labels.
        batch_size (int): Batch size for training.
        epochs (int): Number of epochs for training.

    Returns:
        history: Training history object.
    """
    callbacks = get_callbacks()

    history = model.fit(x_train, y_train,
                        validation_data=(x_val, y_val),
                        batch_size=batch_size,
                        epochs=epochs,
                        callbacks=callbacks,
                        verbose=1)

    return history


def evaluate_model(model, x_test, y_test):
    """
    Evaluates the trained model on the test data.

    Args:
        model (Sequential): Trained Keras model.
        x_test (numpy.array): Test images.
        y_test (numpy.array): Test labels.

    Returns:
        loss (float): Test loss.
        accuracy (float): Test accuracy.
    """
    loss, accuracy = model.evaluate(x_test, y_test, verbose=0)
    print(f"Test loss: {loss:.4f}")
    print(f"Test accuracy: {accuracy:.4f}")

    return loss, accuracy


def main():
    # Load and preprocess data
    x_train, y_train, x_test, y_test = load_and_preprocess_data()

    # Split the training data into training and validation sets
    x_val, y_val = x_train[-10000:], y_train[-10000:]
    x_train, y_train = x_train[:-10000], y_train[:-10000]

    # Build the model
    model = build_model(INPUT_SHAPE, NUM_CLASSES)

    # Train the model
    train_model(model, x_train, y_train, x_val, y_val, BATCH_SIZE, EPOCHS)

    # Evaluate the model on the test set
    evaluate_model(model, x_test, y_test)


if __name__ == "__main__":
    main()


2024-08-31 17:49:24.856148: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-08-31 17:49:25.595461: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


170498071/170498071 [==============================] - 44s 0us/step


2024-08-31 17:50:15.232648: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:266] failed call to cuInit: CUDA_ERROR_COMPAT_NOT_SUPPORTED_ON_DEVICE: forward compatibility was attempted on non supported HW
2024-08-31 17:50:15.232694: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:168] retrieving CUDA diagnostic information for host: cs
2024-08-31 17:50:15.232704: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:175] hostname: cs
2024-08-31 17:50:15.232875: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:199] libcuda reported version is: 535.183.1
2024-08-31 17:50:15.232907: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:203] kernel reported version is: 470.256.2
2024-08-31 17:50:15.232916: E tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:312] kernel version 470.256.2 does not match DSO version 535.183.1 -- cannot find working devices in this configuration


Epoch 1/50


2024-08-31 17:50:16.161740: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 491520000 exceeds 10% of free system memory.


624/625 [============================>.] - ETA: 0s - loss: 1.6680 - accuracy: 0.3811

2024-08-31 17:50:42.454223: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 122880000 exceeds 10% of free system memory.


625/625 [==============================] - 27s 42ms/step - loss: 1.6676 - accuracy: 0.3812 - val_loss: 1.3314 - val_accuracy: 0.5190 - lr: 0.0010
Epoch 2/50
625/625 [==============================] - 24s 39ms/step - loss: 1.2966 - accuracy: 0.5322 - val_loss: 1.1002 - val_accuracy: 0.6150 - lr: 0.0010
Epoch 3/50
625/625 [==============================] - 24s 38ms/step - loss: 1.1541 - accuracy: 0.5896 - val_loss: 0.9960 - val_accuracy: 0.6439 - lr: 0.0010
Epoch 4/50
625/625 [==============================] - 25s 39ms/step - loss: 1.0469 - accuracy: 0.6285 - val_loss: 0.9194 - val_accuracy: 0.6751 - lr: 0.0010
Epoch 5/50
625/625 [==============================] - 24s 39ms/step - loss: 0.9642 - accuracy: 0.6582 - val_loss: 0.8857 - val_accuracy: 0.6890 - lr: 0.0010
Epoch 6/50
625/625 [==============================] - 24s 39ms/step - loss: 0.9113 - accuracy: 0.6794 - val_loss: 0.7970 - val_accuracy: 0.7209 - lr: 0.0010
Epoch 7/50
625/625 [==============================] - 24s 39ms/step -

2024-08-31 18:11:12.047326: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 122880000 exceeds 10% of free system memory.


Test loss: 0.6243
Test accuracy: 0.7964


In [2]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.utils import to_categorical

# Ensure TensorFlow is set up correctly
assert tf.__version__ >= "2.0", "This code requires TensorFlow 2.0 or higher."

# Constants
INPUT_SHAPE = (32, 32, 3)
NUM_CLASSES = 10
BATCH_SIZE = 64
EPOCHS = 50
LEARNING_RATE = 0.001

def load_and_preprocess_data():
    """
    Loads the CIFAR-10 dataset, converts images to the frequency domain, and preprocesses it for training.
    """
    (x_train, y_train), (x_test, y_test) = cifar10.load_data()

    # Convert images to frequency domain using Fourier Transform
    x_train = np.fft.fft2(x_train, axes=(1, 2))
    x_test = np.fft.fft2(x_test, axes=(1, 2))

    # Take the magnitude of the Fourier coefficients
    x_train = np.abs(x_train)
    x_test = np.abs(x_test)

    # Normalize the images to [0, 1] range
    x_train = x_train.astype('float32') / np.max(x_train)
    x_test = x_test.astype('float32') / np.max(x_test)

    # One-hot encode the labels
    y_train = to_categorical(y_train, NUM_CLASSES)
    y_test = to_categorical(y_test, NUM_CLASSES)

    return x_train, y_train, x_test, y_test


def build_model(input_shape, num_classes):
    """
    Builds a CNN model for CIFAR-10 classification.

    Args:
        input_shape (tuple): Shape of the input images.
        num_classes (int): Number of classes in the dataset.

    Returns:
        model (Sequential): Compiled Keras model.
    """
    model = Sequential([
        Conv2D(32, (3, 3), activation='relu', padding='same', input_shape=input_shape),
        MaxPooling2D((2, 2)),
        Dropout(0.25),

        Conv2D(64, (3, 3), activation='relu', padding='same'),
        MaxPooling2D((2, 2)),
        Dropout(0.25),

        Conv2D(128, (3, 3), activation='relu', padding='same'),
        MaxPooling2D((2, 2)),
        Dropout(0.25),

        Flatten(),

        Dense(512, activation='relu'),
        Dropout(0.5),
        
        Dense(num_classes, activation='softmax')
    ])

    model.compile(optimizer=Adam(learning_rate=LEARNING_RATE),
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])

    return model


def get_callbacks():
    """
    Returns a list of callbacks for training.

    Returns:
        list: List of Keras callbacks.
    """
    early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
    reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=0.00001)
    
    return [early_stopping, reduce_lr]


def train_model(model, x_train, y_train, x_val, y_val, batch_size, epochs):
    """
    Trains the CNN model on the training data.

    Args:
        model (Sequential): Keras model to be trained.
        x_train (numpy.array): Training images.
        y_train (numpy.array): Training labels.
        x_val (numpy.array): Validation images.
        y_val (numpy.array): Validation labels.
        batch_size (int): Batch size for training.
        epochs (int): Number of epochs for training.

    Returns:
        history: Training history object.
    """
    callbacks = get_callbacks()

    history = model.fit(x_train, y_train,
                        validation_data=(x_val, y_val),
                        batch_size=batch_size,
                        epochs=epochs,
                        callbacks=callbacks,
                        verbose=1)

    return history


def evaluate_model(model, x_test, y_test):
    """
    Evaluates the trained model on the test data.

    Args:
        model (Sequential): Trained Keras model.
        x_test (numpy.array): Test images.
        y_test (numpy.array): Test labels.

    Returns:
        loss (float): Test loss.
        accuracy (float): Test accuracy.
    """
    loss, accuracy = model.evaluate(x_test, y_test, verbose=0)
    print(f"Test loss: {loss:.4f}")
    print(f"Test accuracy: {accuracy:.4f}")

    return loss, accuracy


def main():
    # Load and preprocess data
    x_train, y_train, x_test, y_test = load_and_preprocess_data()

    # Split the training data into training and validation sets
    x_val, y_val = x_train[-10000:], y_train[-10000:]
    x_train, y_train = x_train[:-10000], y_train[:-10000]

    # Build the model
    model = build_model(INPUT_SHAPE, NUM_CLASSES)

    # Train the model
    train_model(model, x_train, y_train, x_val, y_val, BATCH_SIZE, EPOCHS)

    # Evaluate the model on the test set
    evaluate_model(model, x_test, y_test)


if __name__ == "__main__":
    main()


Epoch 1/50


2024-08-31 22:24:30.436695: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 491520000 exceeds 10% of free system memory.


624/625 [============================>.] - ETA: 0s - loss: 1.9130 - accuracy: 0.2908

2024-08-31 22:24:55.287928: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 122880000 exceeds 10% of free system memory.


625/625 [==============================] - 26s 40ms/step - loss: 1.9129 - accuracy: 0.2908 - val_loss: 1.6891 - val_accuracy: 0.3804 - lr: 0.0010
Epoch 2/50
625/625 [==============================] - 25s 41ms/step - loss: 1.6980 - accuracy: 0.3808 - val_loss: 1.5767 - val_accuracy: 0.4265 - lr: 0.0010
Epoch 3/50
625/625 [==============================] - 27s 43ms/step - loss: 1.6397 - accuracy: 0.4027 - val_loss: 1.5295 - val_accuracy: 0.4456 - lr: 0.0010
Epoch 4/50
625/625 [==============================] - 25s 40ms/step - loss: 1.6096 - accuracy: 0.4139 - val_loss: 1.5079 - val_accuracy: 0.4535 - lr: 0.0010
Epoch 5/50
625/625 [==============================] - 26s 41ms/step - loss: 1.5894 - accuracy: 0.4211 - val_loss: 1.4918 - val_accuracy: 0.4525 - lr: 0.0010
Epoch 6/50
625/625 [==============================] - 24s 39ms/step - loss: 1.5635 - accuracy: 0.4318 - val_loss: 1.5183 - val_accuracy: 0.4516 - lr: 0.0010
Epoch 7/50
625/625 [==============================] - 25s 39ms/step -

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.models import Model
from tensorflow.keras.layers import (Input, Conv2D, MaxPooling2D, Flatten,
                                     Dense, Dropout, concatenate)
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical

# Constants
INPUT_SHAPE = (32, 32, 3)
NUM_CLASSES = 10
BATCH_SIZE = 64
EPOCHS = 50
LEARNING_RATE = 0.001

def load_and_preprocess_data():
    (x_train, y_train), (x_test, y_test) = cifar10.load_data()

    # Normalize spatial data
    x_train_spatial = x_train.astype('float32') / 255.0
    x_test_spatial = x_test.astype('float32') / 255.0

    # Convert to frequency domain
    x_train_freq = np.fft.fft2(x_train, axes=(1, 2))
    x_test_freq = np.fft.fft2(x_test, axes=(1, 2))

    # Shift zero-frequency component to center
    x_train_freq = np.fft.fftshift(x_train_freq, axes=(1, 2))
    x_test_freq = np.fft.fftshift(x_test_freq, axes=(1, 2))

    # Take magnitude and normalize
    x_train_freq = np.log(np.abs(x_train_freq) + 1e-8)
    x_test_freq = np.log(np.abs(x_test_freq) + 1e-8)

    x_train_freq = x_train_freq / np.max(x_train_freq)
    x_test_freq = x_test_freq / np.max(x_test_freq)

    # One-hot encode labels
    y_train = to_categorical(y_train, NUM_CLASSES)
    y_test = to_categorical(y_test, NUM_CLASSES)

    return (x_train_spatial, x_train_freq, y_train), (x_test_spatial, x_test_freq, y_test)

def build_hybrid_model(input_shape, num_classes):
    # Spatial input branch
    spatial_input = Input(shape=input_shape, name='spatial_input')
    x1 = Conv2D(32, (3, 3), activation='relu', padding='same')(spatial_input)
    x1 = MaxPooling2D((2, 2))(x1)
    x1 = Dropout(0.25)(x1)

    x1 = Conv2D(64, (3, 3), activation='relu', padding='same')(spatial_input)
    x1 = MaxPooling2D((2, 2))(x1)
    x1 = Dropout(0.25)(x1)

    x1 = Conv2D(128, (3, 3), activation='relu', padding='same')(spatial_input)
    x1 = MaxPooling2D((2, 2))(x1)
    x1 = Dropout(0.25)(x1)
    
    x1 = Flatten()(x1)

    # Frequency input branch
    freq_input = Input(shape=input_shape, name='freq_input')
    x2 = Conv2D(32, (3, 3), activation='relu', padding='same')(freq_input)
    x2 = MaxPooling2D((2, 2))(x2)
    x2 = Dropout(0.25)(x2)

    x2 = Conv2D(64, (3, 3), activation='relu', padding='same')(freq_input)
    x2 = MaxPooling2D((2, 2))(x2)
    x2 = Dropout(0.25)(x2)

    x2 = Conv2D(128, (3, 3), activation='relu', padding='same')(freq_input)
    x2 = MaxPooling2D((2, 2))(x2)
    x2 = Dropout(0.25)(x2)
    
    x2 = Flatten()(x2)

    # Combine branches
    combined = concatenate([x1, x2])

    # Fully connected layers
    x = Dense(512, activation='relu')(combined)
    x = Dropout(0.5)(x)
    output = Dense(num_classes, activation='softmax')(x)

    model = Model(inputs=[spatial_input, freq_input], outputs=output)

    model.compile(optimizer=Adam(learning_rate=LEARNING_RATE),
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])

    return model

def main():
    (x_train_spatial, x_train_freq, y_train), (x_test_spatial, x_test_freq, y_test) = load_and_preprocess_data()

    # Split validation data
    x_val_spatial = x_train_spatial[-10000:]
    x_val_freq = x_train_freq[-10000:]
    y_val = y_train[-10000:]

    x_train_spatial = x_train_spatial[:-10000]
    x_train_freq = x_train_freq[:-10000]
    y_train = y_train[:-10000]

    model = build_hybrid_model(INPUT_SHAPE, NUM_CLASSES)

    model.fit({'spatial_input': x_train_spatial, 'freq_input': x_train_freq},
              y_train,
              validation_data=({'spatial_input': x_val_spatial, 'freq_input': x_val_freq}, y_val),
              epochs=EPOCHS,
              batch_size=BATCH_SIZE,
              verbose=1)

    loss, accuracy = model.evaluate({'spatial_input': x_test_spatial, 'freq_input': x_test_freq}, y_test, verbose=0)
    print(f"Test Loss: {loss:.4f}")
    print(f"Test Accuracy: {accuracy:.4f}")

if __name__ == "__main__":
    main()


2024-08-31 22:45:46.747298: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-08-31 22:45:47.536354: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2024-08-31 22:45:58.727522: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:266] failed call to cuInit: CUDA_ERROR_COMPAT_NOT_SUPPORTED_ON_DEVICE: forward compatibility was attempted on non supported HW
2024-08-31 22:45:58.727561: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:168] retrieving CUDA diagnostic information for host: cs
2024-08-31 22:45:58.727570: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:175] hostname: cs
2024-08-31 22:45:58.727717: I tensorflow/compiler/xla/stream_executor/cuda/cuda_di

Epoch 1/50
625/625 [==============================] - 257s 410ms/step - loss: 1.6226 - accuracy: 0.4286 - val_loss: 1.2773 - val_accuracy: 0.5475
Epoch 2/50
625/625 [==============================] - 255s 408ms/step - loss: 1.2603 - accuracy: 0.5495 - val_loss: 1.1308 - val_accuracy: 0.6068
Epoch 3/50
625/625 [==============================] - 254s 406ms/step - loss: 1.1239 - accuracy: 0.5981 - val_loss: 1.0656 - val_accuracy: 0.6253
Epoch 4/50
625/625 [==============================] - 252s 404ms/step - loss: 1.0270 - accuracy: 0.6387 - val_loss: 1.0414 - val_accuracy: 0.6388
Epoch 5/50
625/625 [==============================] - 256s 409ms/step - loss: 0.9432 - accuracy: 0.6671 - val_loss: 0.9797 - val_accuracy: 0.6657
Epoch 6/50
625/625 [==============================] - 257s 412ms/step - loss: 0.8719 - accuracy: 0.6900 - val_loss: 0.9832 - val_accuracy: 0.6575
Epoch 7/50
625/625 [==============================] - 273s 437ms/step - loss: 0.8008 - accuracy: 0.7166 - val_loss: 0.9753 -

In [2]:
import os
import urllib.request
import zipfile
import tarfile

# Base URL for the Places365 dataset
BASE_URL = "http://data.csail.mit.edu/places/places365/"

# Paths for the downloaded files
DATA_DIR = "./places365"
TRAINING_TAR = "train_large_places365standard.tar"
VALIDATION_TAR = "val_large.tar"
METADATA_FILE = "filelist_places365-standard.tar"

# Ensure the data directory exists
if not os.path.exists(DATA_DIR):
    os.makedirs(DATA_DIR)

def download_file(url, destination):
    """
    Downloads a file from a URL to a destination on the local filesystem.
    """
    if not os.path.exists(destination):
        print(f"Downloading {destination}...")
        urllib.request.urlretrieve(url, destination)
        print(f"Downloaded {destination}")
    else:
        print(f"{destination} already exists, skipping download.")

def extract_tar_file(tar_path, extract_path):
    """
    Extracts a tar file to a specified directory.
    """
    print(f"Extracting {tar_path}...")
    with tarfile.open(tar_path, 'r') as tar:
        tar.extractall(path=extract_path)
    print(f"Extracted {tar_path} to {extract_path}")

def download_and_extract_places365():
    """
    Downloads and extracts the Places365 dataset.
    """
    # Download training, validation, and metadata files
    download_file(BASE_URL + TRAINING_TAR, os.path.join(DATA_DIR, TRAINING_TAR))
    download_file(BASE_URL + VALIDATION_TAR, os.path.join(DATA_DIR, VALIDATION_TAR))
    download_file(BASE_URL + METADATA_FILE, os.path.join(DATA_DIR, METADATA_FILE))

    # Extract the tar files
    extract_tar_file(os.path.join(DATA_DIR, TRAINING_TAR), os.path.join(DATA_DIR, "train"))
    extract_tar_file(os.path.join(DATA_DIR, VALIDATION_TAR), os.path.join(DATA_DIR, "val"))
    extract_tar_file(os.path.join(DATA_DIR, METADATA_FILE), DATA_DIR)

def main():
    download_and_extract_places365()
    print("Places365 dataset download and extraction complete.")

if __name__ == "__main__":
    main()


KeyboardInterrupt: 

In [1]:
import tensorflow as tf
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras import layers
import numpy as np

# Constants
INPUT_SHAPE = (32, 32, 3)
BATCH_SIZE = 64
EPOCHS = 50
LEARNING_RATE = 0.001

def load_and_preprocess_data():
    """
    Loads the CIFAR-10 dataset and preprocesses it for training.
    """
    (x_train, _), (x_test, _) = cifar10.load_data()

    # Normalize the images to [0, 1] range
    x_train = x_train.astype('float32') / 255.0
    x_test = x_test.astype('float32') / 255.0

    # Compute FFT of the training and testing images
    y_train = np.fft.fft2(x_train, axes=(1, 2))
    y_test = np.fft.fft2(x_test, axes=(1, 2))

    # Split real and imaginary parts for the FFT result
    y_train_real = np.real(y_train)
    y_train_imag = np.imag(y_train)
    
    y_test_real = np.real(y_test)
    y_test_imag = np.imag(y_test)

    return x_train, y_train_real, y_train_imag, x_test, y_test_real, y_test_imag


# Custom layer to predict the FFT (real and imaginary parts)
class PredictFFTLayer(layers.Layer):
    def __init__(self):
        super(PredictFFTLayer, self).__init__()

    def call(self, inputs):
        # Compute the 2D FFT of the input image
        fft = tf.signal.fft2d(tf.cast(inputs, tf.complex64))
        fft_real = tf.math.real(fft)
        fft_imag = tf.math.imag(fft)
        return fft_real, fft_imag


def build_model(input_shape):
    """
    Builds a CNN model for predicting the FFT of input images.

    Args:
        input_shape (tuple): Shape of the input images.

    Returns:
        model (Sequential): Compiled Keras model.
    """
    inputs = layers.Input(shape=input_shape)

    # Simple CNN model to process the image
    x = Conv2D(32, (3, 3), activation='relu', padding='same')(inputs)
    x = MaxPooling2D((2, 2))(x)
    x = Conv2D(64, (3, 3), activation='relu', padding='same')(x)
    x = MaxPooling2D((2, 2))(x)
    x = Conv2D(128, (3, 3), activation='relu', padding='same')(x)
    
    # Custom layer to predict real and imaginary parts of the FFT
    fft_real, fft_imag = PredictFFTLayer()(x)

    # Create model with real and imaginary FFT outputs
    model = tf.keras.models.Model(inputs=inputs, outputs=[fft_real, fft_imag])

    model.compile(optimizer=Adam(learning_rate=LEARNING_RATE),
                  loss='mse',  # Mean Squared Error for real and imaginary parts
                  metrics=['accuracy'])

    return model


def train_model(model, x_train, y_train_real, y_train_imag, x_val, y_val_real, y_val_imag, batch_size, epochs):
    """
    Trains the CNN model on the training data.

    Args:
        model (Sequential): Keras model to be trained.
        x_train (numpy.array): Training images.
        y_train_real (numpy.array): Real part of FFT for training images.
        y_train_imag (numpy.array): Imaginary part of FFT for training images.
        x_val (numpy.array): Validation images.
        y_val_real (numpy.array): Real part of FFT for validation images.
        y_val_imag (numpy.array): Imaginary part of FFT for validation images.
        batch_size (int): Batch size for training.
        epochs (int): Number of epochs for training.

    Returns:
        history: Training history object.
    """
    callbacks = [EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True),
                 ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=0.00001)]

    history = model.fit(x_train, [y_train_real, y_train_imag],
                        validation_data=(x_val, [y_val_real, y_val_imag]),
                        batch_size=batch_size,
                        epochs=epochs,
                        callbacks=callbacks,
                        verbose=1)

    return history


def main():
    # Load and preprocess data
    x_train, y_train_real, y_train_imag, x_test, y_test_real, y_test_imag = load_and_preprocess_data()

    # Split the training data into training and validation sets
    x_val, y_val_real, y_val_imag = x_train[-10000:], y_train_real[-10000:], y_train_imag[-10000:]
    x_train, y_train_real, y_train_imag = x_train[:-10000], y_train_real[:-10000], y_train_imag[:-10000]

    # Build the model
    model = build_model(INPUT_SHAPE)

    # Train the model
    train_model(model, x_train, y_train_real, y_train_imag, x_val, y_val_real, y_val_imag, BATCH_SIZE, EPOCHS)

    # Evaluate the model on the test set
    test_loss_real, test_loss_imag = model.evaluate(x_test, [y_test_real, y_test_imag], verbose=0)
    print(f"Test loss (Real): {test_loss_real:.4f}")
    print(f"Test loss (Imaginary): {test_loss_imag:.4f}")


if __name__ == "__main__":
    main()


2024-09-07 14:39:37.650200: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-09-07 14:39:38.830186: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2024-09-07 14:39:51.937419: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:266] failed call to cuInit: CUDA_ERROR_COMPAT_NOT_SUPPORTED_ON_DEVICE: forward compatibility was attempted on non supported HW
2024-09-07 14:39:51.938427: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:168] retrieving CUDA diagnostic information for host: cs
2024-09-07 14:39:51.938490: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:175] hostname: cs
2024-09-07 14:39:51.938930: I tensorflow/compiler/xla/stream_executor/cuda/cuda_di

Epoch 1/50


2024-09-07 14:39:55.685420: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 491520000 exceeds 10% of free system memory.


ValueError: in user code:

    File "/home/cs/anaconda3/lib/python3.9/site-packages/keras/engine/training.py", line 1284, in train_function  *
        return step_function(self, iterator)
    File "/home/cs/anaconda3/lib/python3.9/site-packages/keras/engine/training.py", line 1268, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/home/cs/anaconda3/lib/python3.9/site-packages/keras/engine/training.py", line 1249, in run_step  **
        outputs = model.train_step(data)
    File "/home/cs/anaconda3/lib/python3.9/site-packages/keras/engine/training.py", line 1051, in train_step
        loss = self.compute_loss(x, y, y_pred, sample_weight)
    File "/home/cs/anaconda3/lib/python3.9/site-packages/keras/engine/training.py", line 1109, in compute_loss
        return self.compiled_loss(
    File "/home/cs/anaconda3/lib/python3.9/site-packages/keras/engine/compile_utils.py", line 265, in __call__
        loss_value = loss_obj(y_t, y_p, samp

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.optimizers import Adam
import numpy as np
from tensorflow.keras.datasets import cifar10

# Constants
INPUT_SHAPE = (32, 32, 3)
BATCH_SIZE = 32
EPOCHS = 50
LEARNING_RATE = 0.001

def load_and_preprocess_data():
    """
    Loads the CIFAR-10 dataset and preprocesses it for training.
    Returns images and their FFTs (real and imaginary parts).
    """
    (x_train, _), (x_test, _) = cifar10.load_data()

    # Normalize the images to [0, 1] range
    x_train = x_train.astype('float32') / 255.0
    x_test = x_test.astype('float32') / 255.0

    # Compute FFT of the training and testing images
    y_train = np.fft.fft2(x_train, axes=(1, 2))
    y_test = np.fft.fft2(x_test, axes=(1, 2))

    # Split real and imaginary parts for the FFT result
    y_train_real = np.real(y_train)
    y_train_imag = np.imag(y_train)
    
    y_test_real = np.real(y_test)
    y_test_imag = np.imag(y_test)

    # Concatenate real and imaginary parts along the channel axis
    y_train_fft = np.concatenate([y_train_real, y_train_imag], axis=-1)  # Shape: (batch, 32, 32, 6)
    y_test_fft = np.concatenate([y_test_real, y_test_imag], axis=-1)

    return x_train, y_train_fft, x_test, y_test_fft

def build_model(input_shape):
    """
    Builds a CNN model to predict the FFT (real and imaginary parts) of input images.
    """
    model = models.Sequential([
        # CNN layers
        layers.Conv2D(32, (3, 3), activation='relu', padding='same', input_shape=input_shape),
        layers.Conv2D(64, (3, 3), activation='relu', padding='same'),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(128, (3, 3), activation='relu', padding='same'),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(128, (3, 3), activation='relu', padding='same'),
        
        # Upsample to original size
        layers.UpSampling2D((2, 2)),
        layers.Conv2D(64, (3, 3), activation='relu', padding='same'),
        layers.UpSampling2D((2, 2)),
        
        # Output layer with 6 channels (3 for real, 3 for imaginary parts of the FFT)
        layers.Conv2D(6, (3, 3), activation='linear', padding='same')
    ])

    model.compile(optimizer=Adam(learning_rate=LEARNING_RATE),
                  loss='mse',  # Mean Squared Error for real and imaginary parts
                  metrics=['accuracy'])

    return model


def train_model(model, x_train, y_train, x_val, y_val, batch_size, epochs):
    """
    Trains the CNN model on the training data.

    Args:
        model (Sequential): Keras model to be trained.
        x_train (numpy.array): Training images.
        y_train (numpy.array): FFT of training images (real + imaginary).
        x_val (numpy.array): Validation images.
        y_val (numpy.array): FFT of validation images (real + imaginary).
        batch_size (int): Batch size for training.
        epochs (int): Number of epochs for training.

    Returns:
        history: Training history object.
    """
    history = model.fit(x_train, y_train,
                        validation_data=(x_val, y_val),
                        batch_size=batch_size,
                        epochs=epochs,
                        verbose=1)

    return history


def main():
    # Load and preprocess data
    x_train, y_train_fft, x_test, y_test_fft = load_and_preprocess_data()

    # Split the training data into training and validation sets
    x_val, y_val_fft = x_train[-10000:], y_train_fft[-10000:]
    x_train, y_train_fft = x_train[:-10000], y_train_fft[:-10000]

    # Build the model
    model = build_model(INPUT_SHAPE)

    # Train the model
    train_model(model, x_train, y_train_fft, x_val, y_val_fft, BATCH_SIZE, EPOCHS)

    # Evaluate the model on the test set
    test_loss, test_acc = model.evaluate(x_test, y_test_fft, verbose=0)
    print(f"Test loss: {test_loss:.4f}")
    print(f"Test accuracy: {test_acc:.4f}")


if __name__ == "__main__":
    main()


2024-09-07 14:49:00.585170: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-09-07 14:49:01.588577: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2024-09-07 14:49:08.621267: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:266] failed call to cuInit: CUDA_ERROR_COMPAT_NOT_SUPPORTED_ON_DEVICE: forward compatibility was attempted on non supported HW
2024-09-07 14:49:08.621298: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:168] retrieving CUDA diagnostic information for host: cs
2024-09-07 14:49:08.621305: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:175] hostname: cs
2024-09-07 14:49:08.621457: I tensorflow/compiler/xla/stream_executor/cuda/cuda_di

Epoch 1/50


2024-09-07 14:49:09.777079: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 983040000 exceeds 10% of free system memory.


1250/1250 [==============================] - 138s 110ms/step - loss: 27.6343 - accuracy: 0.1783 - val_loss: 22.8926 - val_accuracy: 0.1804
Epoch 2/50
1250/1250 [==============================] - 137s 109ms/step - loss: 22.6252 - accuracy: 0.1817 - val_loss: 22.9492 - val_accuracy: 0.1817
Epoch 3/50
1250/1250 [==============================] - 137s 110ms/step - loss: 21.9949 - accuracy: 0.1833 - val_loss: 21.5809 - val_accuracy: 0.1837
Epoch 4/50
1250/1250 [==============================] - 136s 109ms/step - loss: 21.7067 - accuracy: 0.1842 - val_loss: 20.9990 - val_accuracy: 0.1845
Epoch 5/50
1250/1250 [==============================] - 136s 109ms/step - loss: 21.3909 - accuracy: 0.1845 - val_loss: 21.2162 - val_accuracy: 0.1846
Epoch 6/50
1250/1250 [==============================] - 136s 109ms/step - loss: 21.1039 - accuracy: 0.1849 - val_loss: 20.9632 - val_accuracy: 0.1843
Epoch 7/50
1250/1250 [==============================] - 136s 109ms/step - loss: 20.9084 - accuracy: 0.1852 - va

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, optimizers, applications, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping, ModelCheckpoint
import matplotlib.pyplot as plt
import keras_tuner as kt
import pandas as pd

# Set random seeds for reproducibility
tf.random.set_seed(42)
np.random.seed(42)

# Load CIFAR-10 dataset
(x_train, y_train), (x_test, y_test) = keras.datasets.cifar10.load_data()

# Normalize the data
x_train = x_train.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0

# One-hot encode the labels
y_train = keras.utils.to_categorical(y_train, 10)
y_test = keras.utils.to_categorical(y_test, 10)

# Data augmentation
datagen = ImageDataGenerator(
    horizontal_flip=True,
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    fill_mode='nearest'
)
datagen.fit(x_train)

# Hyperparameter optimization function
def build_model(hp):
    base_model = applications.ResNet50(
        weights='imagenet',
        include_top=False,
        input_shape=(32, 32, 3)
    )
    
    # Unfreeze a tunable number of layers in the base model
    max_unfreeze_index = len(base_model.layers) - 1
    unfreeze_at = hp.Int('unfreeze_at', min_value=max_unfreeze_index - 50, max_value=max_unfreeze_index, step=10)
    for i, layer in enumerate(base_model.layers):
        if i >= unfreeze_at:
            if not isinstance(layer, layers.BatchNormalization):
                layer.trainable = True
            else:
                layer.trainable = False
        else:
            layer.trainable = False
            
    model = models.Sequential()
    model.add(base_model)
    model.add(layers.GlobalAveragePooling2D())

    # Add tunable dense layers
    num_dense_layers = hp.Int('num_dense_layers', 0, 3)
    units = hp.Choice('initial_units', values=[256, 512, 1024])
    rate =hp.Float('initial_dropout', min_value=0.2, max_value=0.5, step=0.1)
    for bi in range(num_dense_layers):
        model.add(layers.Dense(units=units, activation='relu'))
        model.add(layers.Dropout(rate = rate))
        units = max(units // 2, 64)
        rate = np.clip(rate - 0.1,a_min=0,a_max=None)

    # Add the output layer
    model.add(layers.Dense(10, activation='softmax'))

    # Compile the model
    model.compile(
        optimizer=optimizers.Adam(learning_rate=hp.Choice('learning_rate', values=[1e-3, 1e-4, 1e-5])),
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )
    return model

# Use Keras Tuner for hyperparameter search
tuner = kt.Hyperband(
    build_model,
    objective='val_accuracy',
    max_epochs=50,
    factor=3,
    directory='hyperparam_tuning',
    project_name='resnet_cifar10_optimization'
)

# Set up callbacks for training
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
lr_scheduler = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, verbose=1)

# Search for the best hyperparameters
tuner.search(
    datagen.flow(x_train, y_train, batch_size=64),
    validation_data=(x_test, y_test),
    epochs=50,
    callbacks=[early_stopping, lr_scheduler],
    steps_per_epoch=x_train.shape[0] // 64,
    verbose=1
)

# Get the best model and evaluate
best_model = tuner.get_best_models(num_models=1)[0]
final_loss, final_accuracy = best_model.evaluate(x_test, y_test, verbose=1)
print(f"Best model test accuracy: {final_accuracy:.4f}")

# Visualize the training history of the best model
def plot_model_history(history):
    plt.figure(figsize=(12, 5))
    plt.subplot(1, 2, 1)
    plt.plot(history.history['accuracy'], label='Training Accuracy')
    plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
    plt.title('Model Accuracy')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.legend()
    
    plt.subplot(1, 2, 2)
    plt.plot(history.history['loss'], label='Training Loss')
    plt.plot(history.history['val_loss'], label='Validation Loss')
    plt.title('Model Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    
    plt.show()

# Train the best model again to get history for visualization
history = best_model.fit(
    datagen.flow(x_train, y_train, batch_size=64),
    validation_data=(x_test, y_test),
    epochs=50,
    callbacks=[early_stopping, lr_scheduler],
    steps_per_epoch=x_train.shape[0] // 64,
    verbose=1
)

plot_model_history(history)

# Log the best hyperparameters
best_hyperparameters = tuner.get_best_hyperparameters(num_trials=1)[0]
print("Best Hyperparameters:")
print(best_hyperparameters.values)

# Convert hyperparameters to a DataFrame for easy viewing
hp_values_df = pd.DataFrame([best_hyperparameters.values])
tools.display_dataframe_to_user(name="Best Hyperparameters", dataframe=hp_values_df)


Trial 22 Complete [00h 00m 12s]

Best val_accuracy So Far: 0.3027999997138977
Total elapsed time: 00h 18m 04s

Search: Running Trial #23

Value             |Best Value So Far |Hyperparameter
144               |164               |unfreeze_at
3                 |0                 |num_dense_layers
1024              |1024              |initial_units
0.3               |0.4               |initial_dropout
0.001             |0.0001            |learning_rate
2                 |2                 |tuner/epochs
0                 |0                 |tuner/initial_epoch
3                 |3                 |tuner/bracket
0                 |0                 |tuner/round



2024-11-02 14:48:33.147654: W external/local_xla/xla/tsl/framework/bfc_allocator.cc:497] Allocator (GPU_0_bfc) ran out of memory trying to allocate 2.25MiB (rounded to 2359296)requested by op AddV2
If the cause is memory fragmentation maybe the environment variable 'TF_GPU_ALLOCATOR=cuda_malloc_async' will improve the situation. 
Current allocation summary follows.
Current allocation summary follows.
2024-11-02 14:48:33.147791: I external/local_xla/xla/tsl/framework/bfc_allocator.cc:1053] BFCAllocator dump for GPU_0_bfc
2024-11-02 14:48:33.147807: I external/local_xla/xla/tsl/framework/bfc_allocator.cc:1060] Bin (256): 	Total Chunks: 5480, Chunks in use: 5480. 1.34MiB allocated for chunks. 1.34MiB in use in bin. 118.1KiB client-requested in use in bin.
2024-11-02 14:48:33.147817: I external/local_xla/xla/tsl/framework/bfc_allocator.cc:1060] Bin (512): 	Total Chunks: 446, Chunks in use: 446. 224.2KiB allocated for chunks. 224.2KiB in use in bin. 223.0KiB client-requested in use in bin.


RuntimeError: Number of consecutive failures exceeded the limit of 3.
Traceback (most recent call last):
  File "/home/cs/.local/lib/python3.10/site-packages/keras_tuner/src/engine/base_tuner.py", line 274, in _try_run_and_update_trial
    self._run_and_update_trial(trial, *fit_args, **fit_kwargs)
  File "/home/cs/.local/lib/python3.10/site-packages/keras_tuner/src/engine/base_tuner.py", line 239, in _run_and_update_trial
    results = self.run_trial(trial, *fit_args, **fit_kwargs)
  File "/home/cs/.local/lib/python3.10/site-packages/keras_tuner/src/tuners/hyperband.py", line 427, in run_trial
    return super().run_trial(trial, *fit_args, **fit_kwargs)
  File "/home/cs/.local/lib/python3.10/site-packages/keras_tuner/src/engine/tuner.py", line 314, in run_trial
    obj_value = self._build_and_fit_model(trial, *args, **copied_kwargs)
  File "/home/cs/.local/lib/python3.10/site-packages/keras_tuner/src/engine/tuner.py", line 232, in _build_and_fit_model
    model = self._try_build(hp)
  